In [10]:
import json
import os
import shutil

#relative_paths = ['/rod', '/sphere', '/spiral']
#relative_paths = ['/all']

base_path = "../Datasets"
first_image = '/sequence.1/step0.camera.png'

result_base_path = "../Results"   
experiment_names=["/exp-2-1","/exp-2-2"]

In [11]:
#Create the required paths
for e in experiment_names:
    os.makedirs(f"{base_path}{e}", exist_ok=True)
    print(f"The news directorys for the experiment '{base_path}{e}' has been created!")

The news directorys for the experiment '../Datasets/exp-2-1' has been created!
The news directorys for the experiment '../Datasets/exp-2-2' has been created!


In [12]:

relative_paths = {}
for e in experiment_names:
    dir_list = os.listdir(base_path + e)
    relative_paths[e] = ["/" + p  for p in dir_list]

print(relative_paths)

{'/exp-2-1': ['/all'], '/exp-2-2': ['/all']}


In [4]:
def generate_directories(experiment_name):
    #Create the required paths
    os.makedirs(f"{result_base_path}{experiment_name}/train/images", exist_ok=True)
    os.makedirs(f"{result_base_path}{experiment_name}/train/labels", exist_ok=True)
    os.makedirs(f"{result_base_path}{experiment_name}/valid/images", exist_ok=True)
    os.makedirs(f"{result_base_path}{experiment_name}/valid/labels", exist_ok=True)

    print(f"The news directorys for the experiment '{result_base_path}{experiment_name}' has been created!")

In [5]:
# annotations_definitions.json to classes.txt
def read_annotations(image_path, destiny_path):
  print("READ: ", image_path + 'annotation')
  with open(image_path + '/annotation_definitions.json', 'r') as file:
      classes_json_file = json.load(file)

  print("WRITE", destiny_path + "classes.txt")
  with open(destiny_path + "/classes.txt", 'w') as classes_file:
    for classe in classes_json_file['annotationDefinitions'][0]['spec']:
      classes_file.write(
          '%s\n'%(
            classe['label_name'] 
          )
      ) 

In [14]:
from PIL import Image

def generate_files(experiment_name):
    def create_files(input_path, destiny_path, tag):
        print(base_path + experiment_name + relative_paths[experiment_name][0]  + "/"+ first_image)
        with Image.open(base_path + experiment_name + "/" + relative_paths[experiment_name][0] + first_image) as image:
            width, height = image.size
            print(width,height)

        pic_width = width
        pic_height = height
        dir_list = os.listdir(input_path)
        captures_list = [str for str in dir_list if "sequence" in str]

        valid_size = int((len(dir_list)-4)*0.3)


        for capture in captures_list:
            dir2 = os.path.join(input_path, capture, "step0.frame_data.json")
            i = 0

            with open(dir2, 'r') as file:
                big_json_file = json.load(file)

            sequence = big_json_file['sequence']
            data_set = ""

            for picture in big_json_file['captures']:
                # determine if go to valid set or train set
                if(valid_size > 0):
                    valid_size = valid_size -1
                    data_set = "valid"
                else:
                    data_set = "train"
                # Copy the image
                source = os.path.join(input_path, capture, "step0.camera.png")
                destiny_image = os.path.join(destiny_path , data_set, "images" ,f"{tag}_step0.camera1_{sequence}.png")

                try:
                    shutil.copy(source, destiny_image)
                    print("Image file copied successfully.")
                except FileNotFoundError as e:
                    print(f"Error copying image file: {e}")

                # create the label file
                filename = picture['filename'].split('/')[-1]
                filename = filename[:-4] + str(i + 1) + "_" + str(sequence) + '.txt'
                with open(os.path.join(destiny_path, data_set, "labels", f"{tag}_{filename}"), 'w') as annotation_file:
                    for bbox in picture['annotations'][0]['values']:
                        annotation_file.write(
                            '%d %f %f %f %f\n' % (
                            bbox['labelId'],
                                (bbox['origin'][0] + bbox['dimension'][0] / 2) / pic_width,
                                (bbox['origin'][1] + bbox['dimension'][1] / 2) / pic_height,
                                bbox['dimension'][0] / pic_width,
                                bbox['dimension'][1] / pic_height
                            )
                        )
                    print(f"Annotation file '{filename}' created successfully.")
                i += 1


    for i, path in enumerate(relative_paths[experiment_name]):
        create_files(base_path + experiment_name + path, result_base_path+experiment_name, str(i))

In [7]:
def zip_directory(source_dir, zip_filename):
    shutil.make_archive(zip_filename, 'zip', source_dir)

In [15]:
import shutil

for e in experiment_names:
    source_directory = result_base_path + e 
    zip_file_name =  result_base_path + e  
    destination_path = f"shared/{e}.zip"
    generate_directories(e)
    read_annotations(base_path +  e +  relative_paths[e][0], result_base_path + e)
    generate_files(e)
    zip_directory(source_directory, zip_file_name)
    shutil.copy(zip_file_name+'.zip', destination_path)

The news directorys for the experiment '../Results/exp-2-1' has been created!
READ:  ../Datasets/exp-2-1/allannotation
WRITE ../Results/exp-2-1classes.txt
../Datasets/exp-2-1/all//sequence.1/step0.camera.png
640 640
OK2:
Pictures: {'@type': 'type.unity.com/unity.solo.RGBCamera', 'id': 'camera', 'description': '', 'position': [0.0, 0.0, -27.0], 'rotation': [0.0, 0.0, 0.0, 1.0], 'velocity': [0.0, 0.0, 0.0], 'acceleration': [0.0, 0.0, 0.0], 'filename': 'step0.camera.png', 'imageFormat': 'Png', 'dimension': [640.0, 640.0], 'projection': 'Perspective', 'matrix': [11.4300528, 0.0, 0.0, 0.0, 11.4300528, 0.0, 0.0, 0.0, -1.0006001], 'annotations': [{'@type': 'type.unity.com/unity.solo.BoundingBox2DAnnotation', 'id': 'bounding box', 'sensorId': 'camera', 'description': "Produces 2D bounding box annotations for all visible objects that bear a label defined in this labeler's associated label configuration.", 'values': [{'instanceId': 1, 'labelId': 0, 'labelName': 'rod', 'origin': [503.0, 171.0], '